In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%pwd

'/content'

In [3]:
%cd drive/MyDrive/milestone_3_NotAnAGI/project_folders/notebooks/evaluation

/content/drive/MyDrive/milestone_3_NotAnAGI/project_folders/notebooks/evaluation


In [4]:
import sys
MODULES_PATHS = ["../../modules"]
for mpath in MODULES_PATHS:
    if mpath not in sys.path:
        sys.path.append(mpath)

In [6]:
!pip install torch==1.13.1
!pip install -U bitsandbytes
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U git+https://github.com/huggingface/peft.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets
!pip install evaluate
!pip install "git+https://github.com/AIPHES/DiscoScore.git"
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-tneoo_e_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-tneoo_e_
  Resolved https://github.com/huggingface/transformers.git to commit 0c3fdccf2f271fb7c44f6ea6e9f4ee234795f2c5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-j2ut8ts0
  Running command git clone --filter=blob:none --quiet htt

In [45]:
import json
import torch
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from peft import PeftConfig, PeftModel


from datasets_ft import build_hf_dataset
from evaluation import Evaluator

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Preparation

In [55]:
gen_config = {
    # general
    "num_return_sequences": 1,
    "min_length": 16,
    "max_length": 512,

    # sampling
    "top_k": 0,
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.3,

    # against repetition
    "no_repeat_ngram_size": 4,
    "repetition_penalty": 1.5,

    # speed-up
    "use_cache": True
}


def generate_responses(model, tokenizer, all_input_ids, device=torch.device(device)):
    responses = []
    for input_ids in tqdm(all_input_ids):
      gen = model.generate(input_ids=input_ids.to(device).unsqueeze(0), **gen_config).squeeze()
      response = tokenizer.decode(gen, skip_special_tokens=True)
      responses.append(response)

    return responses

In [16]:
evaluator = Evaluator(model_type="bert-base-uncased", device_name=device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Our Assistant Model: Evaluating T5-LARGE (FINETUNED)

In [52]:
datapath = "../../../submission_items/datasets/gen_dataset_NotAnAGI.json"

tokenizer_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
with open(datapath, "r") as reader:
    _, testset = build_hf_dataset(json.load(reader), tokenizer)
    testset = testset.select(range(512))

Map:   0%|          | 0/10647 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10647 [00:00<?, ? examples/s]

In [18]:
model_path = "../../../submission_items/assistant-t5-large-lm"

config = PeftConfig.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
model = PeftModel.from_pretrained(model, model_path)
model = model.to(device)

In [56]:
queries = testset["query"]
responses = generate_responses(
    model=model,
    tokenizer=tokenizer,
    all_input_ids=testset["input_ids"])



  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 1/512 [00:06<51:32,  6.05s/it]

  0%|          | 2/512 [00:10<44:21,  5.22s/it]

  1%|          | 3/512 [00:21<1:04:52,  7.65s/it]

  1%|          | 4/512 [00:24<48:28,  5.73s/it]  

  1%|          | 5/512 [00:27<42:19,  5.01s/it]

  1%|          | 6/512 [00:31<39:33,  4.69s/it]

  1%|▏         | 7/512 [00:38<44:07,  5.24s/it]

  2%|▏         | 8/512 [00:41<39:38,  4.72s/it]

  2%|▏         | 9/512 [00:43<30:54,  3.69s/it]

  2%|▏         | 10/512 [00:45<25:59,  3.11s/it]

  2%|▏         | 11/512 [00:52<38:06,  4.56s/it]

  2%|▏         | 12/512 [01:09<1:09:21,  8.32s/it]

  3%|▎         | 13/512 [01:17<1:08:28,  8.23s/it]

  3%|▎         | 14/512 [01:19<52:20,  6.31s/it]  

  3%|▎         | 15/512 [01:22<43:30,  5.25s/it]

  3%|▎         | 16/512 [01:32<54:14,  6.56s/it]

  3%|▎         | 17/512 [01:47<1:16:04,  9.22s/it]

  4%|▎         | 18/512 [01:56<1:14:29,  9.05s/it]

  4%|▎         | 19/512 [02:01<1:05:31,  7.97s/it]



In [66]:
responses_in_dict = [{"query": q, "model_answer": r} for q, r in zip(queries, responses)]
with open("./t5-large-finetuned-responses.json", "w") as writer:
  json.dump(responses_in_dict, writer, indent=2)

In [ ]:
with open("./t5-large-finetuned-responses.json", "r") as reader:
  queries, responses = zip(*[(x["query"], x["model_answer"]) for x in json.load(reader)])
  queries = list(queries)
  responses = list(responses)

In [57]:
result = evaluator.evaluate(responses, [[q] for q in queries])
result

Already downloaded a model for the 'en' language


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /content/drive/MyDrive/milestone_3_NotAnAGI/project_folders/notebooks/evaluation/../../modules/e │
│ valuation.py:81 in evaluate                                                                      │
│                                                                                                  │
│    78 │   │   assert agg_method in set(["mean", "max", "min", "median", "std"]), f"Invalid agg   │
│    79 │   │                                                                                      │
│    80 │   │   # compute the scores                                                               │
│ ❱  81 │   │   results = self.scores(generations, references_per_generation)                      │
│    82 │   │                                                                                      │
│    83 │   │   # select aggregation function                                                      │
│    84 │   │   agg_fn = None                                                                      │
│                                                                                                  │
│ /content/drive/MyDrive/milestone_3_NotAnAGI/project_folders/notebooks/evaluation/../../modules/e │
│ valuation.py:45 in scores                                                                        │
│                                                                                                  │
│    42 │   │   disco_scores_plus= []                                                              │
│    43 │   │   for gen, refs in zip(generations, references_per_generation):                      │
│    44 │   │   │   disco_scores_plus_result = {                                                   │
│ ❱  45 │   │   │   │   "ds_focus_nn": self.disco_scorer.DS_Focus_NN(gen, refs),                   │
│    46 │   │   │   │   "ds_sent_nn": self.disco_scorer.DS_SENT_NN(gen, refs),                     │
│    47 │   │   │   │   "rc": self.disco_scorer.RC(gen, refs),                                     │
│    48 │   │   │   │   "lc": self.disco_scorer.LC(gen, refs),                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/disco_score/scorer.py:24 in DS_Focus_NN                  │
│                                                                                                  │
│   21 │   │   return discourse.LexicalChain(sys, ref)                                             │
│   22 │                                                                                           │
│   23 │   def DS_Focus_NN(self, sys, ref):                                                        │
│ ❱ 24 │   │   return discourse.DS_Focus(self.model, self.tokenizer, sys, ref, is_semantic_enti    │
│   25 │                                                                                           │
│   26 │   def DS_Focus_Entity(self, sys, ref):                                                    │
│   27 │   │   return discourse.DS_Focus(self.model, self.tokenizer, sys, ref, is_semantic_enti    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/disco_score/metrics/discourse.py:126 in DS_Focus         │
│                                                                                                  │
│   123 │   from disco_score.metrics.focus_diff import get_embeddings                              │
│   124 │                                                                                          │
│   125 │   scores = []                                      

### Evaluating T5-BASE (FINETUNED)

In [65]:
tokenizer_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
with open(datapath, "r") as reader:
    _, testset = build_hf_dataset(json.load(reader), tokenizer)
    testset = testset.select(range(512))

Map:   0%|          | 0/10647 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10647 [00:00<?, ? examples/s]

In [67]:
model_path = "../../models/t5-finetuned/t5-base"

config = PeftConfig.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
model = PeftModel.from_pretrained(model, model_path)
model = model.to(device)

In [69]:
queries = testset["query"]
responses = generate_responses(
    model=model,
    tokenizer=tokenizer,
    all_input_ids=testset["input_ids"])

100%|██████████| 512/512 [42:10<00:00,  4.94s/it]


In [70]:
responses_in_dict = [{"query": q, "model_answer": r} for q, r in zip(queries, responses)]
with open("./t5-base-finetuned-responses.json", "w") as writer:
  json.dump(responses_in_dict, writer, indent=2)

In [ ]:
with open("./t5-base-finetuned-responses.json", "r") as reader:
  queries, responses = zip(*[(x["query"], x["model_answer"]) for x in json.load(reader)])
  queries = list(queries)
  responses = list(responses)

In [ ]:
result = evaluator.evaluate(responses, [[q] for q in queries])
result

### Evaluating T5-BASE (FINETUNED + PPO)

In [71]:
model_path = "../../models/t5-base-finetuned-ppo/full-training"

config = PeftConfig.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
model = PeftModel.from_pretrained(model, model_path)
model = model.to(device)

In [72]:
queries = testset["query"]
responses = generate_responses(
    model=model,
    tokenizer=tokenizer,
    all_input_ids=testset["input_ids"])

100%|██████████| 512/512 [17:57<00:00,  2.10s/it]


In [73]:
responses_in_dict = [{"query": q, "model_answer": r} for q, r in zip(queries, responses)]
with open("./t5-base-finetuned-ppo-responses.json", "w") as writer:
  json.dump(responses_in_dict, writer, indent=2)

In [ ]:
with open("./t5-base-finetuned-ppo-responses.json", "r") as reader:
  queries, responses = zip(*[(x["query"], x["model_answer"]) for x in json.load(reader)])
  queries = list(queries)
  responses = list(responses)

In [ ]:
result = evaluator.evaluate(responses, [[q] for q in queries])
result

### Evaluating GPT2 (PRETRAINED)

In [86]:
tokenizer_id = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

with open(datapath, "r") as reader:
    _, testset = build_hf_dataset(json.load(reader), tokenizer)
    testset = testset.select(range(512))

Map:   0%|          | 0/10647 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10647 [00:00<?, ? examples/s]

In [87]:
model_path = "gpt2-large"

model = AutoModelForCausalLM.from_pretrained(model_path)
model = model.to(device)

In [100]:
def gpt2_generate_responses(model, tokenizer, all_input_ids, all_queries, device=device):
  attn_masks = [torch.ones_like(input_ids) for input_ids in all_input_ids]

  responses = []
  for query, input_ids, attn_mask in tqdm(list(zip(all_queries, all_input_ids, attn_masks))):
      gen = model.generate(input_ids=input_ids.to(device).unsqueeze(0), attention_mask=attn_mask.to(device).unsqueeze(0), **gen_config).squeeze()
      response = tokenizer.decode(gen, skip_special_tokens=True)
      response = response.split(query)[-1].strip()
      responses.append(response)

  return responses

In [ ]:
queries = testset["query"]
responses = gpt2_generate_responses(
    model=model,
    tokenizer=tokenizer,
    all_input_ids=testset["input_ids"],
    all_queries=queries)

  1%|          | 4/512 [00:31<1:02:01,  7.33s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 605, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  3%|▎         | 17/512 [02:03<45:46,  5.55s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 617, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  7%|▋         | 37/512 [05:00<1:22:48, 10.46s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 741, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
 11%|█         | 55/512 [07:43<1:22:00, 10.77s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 542, but `max_length` is se

In [ ]:
responses_in_dict = [{"query": q, "model_answer": r} for q, r in zip(queries, responses)]
with open("./gpt2-large-responses.json", "w") as writer:
  json.dump(responses_in_dict, writer, indent=2)

In [ ]:
with open("./gpt2-large-responses.json", "r") as reader:
  queries, responses = zip(*[(x["query"], x["model_answer"]) for x in json.load(reader)])
  queries = list(queries)
  responses = list(responses)

In [ ]:
result = evaluator.evaluate(responses, [[q] for q in queries])
result